<a href="https://colab.research.google.com/github/Srija-Swarna/Assignment-3---Individual-project/blob/main/Assignment_3_Individual_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code starts by importing a set of necessary libraries and modules. numpy is used for numerical operations, datasets from the Hugging Face library is utilized to load and handle datasets, and transformers provides tools for using BERT (Bidirectional Encoder Representations from Transformers), a powerful transformer-based model. Specifically, BertTokenizer and BertForSequenceClassification are used for tokenizing text and for defining the BERT model with a sequence classification task. Trainer and TrainingArguments are utilities for training the model, while DataCollatorWithPadding helps in dynamically padding the input data. The pipeline function is used for easy application of the model to new inputs. For evaluation, accuracy_score from sklearn.metrics is used to calculate accuracy

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
# Importing Necessary Libraries
import numpy as np
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

The dataset "dvilasuero/ag_news_error_analysis" is loaded using the load_dataset function. This dataset contains text inputs and their associated annotations, which the model will use for training and evaluation. The structure and content of the dataset are printed to the console, allowing for an initial inspection of the data.

In [ ]:
# Loading and Printing the Dataset
dataset = load_dataset("dvilasuero/ag_news_error_analysis")
print("Dataset:", dataset)

The BERT tokenizer and model are loaded from the pre-trained 'bert-base-uncased' model. The tokenizer is responsible for converting raw text into token IDs that the model can process. The model is initialized for a sequence classification task with four labels, indicating it will classify inputs into one of four possible categories.

In [ ]:
# Loading the Tokenizer and the Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

A function preprocess_function is defined to tokenize the inputs from the dataset. This function uses the BERT tokenizer to encode the text data, ensuring each input is padded and truncated to a maximum length of 256 tokens. The function also sets the 'labels' field, which corresponds to the annotations in the dataset, making sure the labels are available during training. The dataset is then tokenized and converted into a format suitable for PyTorch tensors, with the necessary columns (input_ids, attention_mask, labels) retained.

In [ ]:
# Tokenizing the Dataset
def preprocess_function(examples):
    tokenized = tokenizer(examples['inputs'], padding="max_length", truncation=True, max_length=256)
    tokenized['labels'] = examples['annotation']  # Ensure 'labels' are included
    return tokenized

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

The training portion of the tokenized dataset is further split into a smaller training set and a validation set using the train_test_split method. This step is crucial for evaluating the model's performance on unseen data (validation set) during training, helping to monitor for overfitting and to tune hyperparameters.

In [ ]:
# Splitting the Training dataset into Train and Validation sets
small_train_dataset = tokenized_datasets['train'].train_test_split(test_size=0.1)
train_dataset = small_train_dataset['train']
eval_dataset = small_train_dataset['test']

A DataCollatorWithPadding is instantiated, which pads the inputs dynamically to the same length within a batch. This is essential for efficient processing in batches during training. The TrainingArguments object is configured with various parameters: the directory for saving model outputs (output_dir), evaluation strategy (epoch to evaluate after each epoch), learning rate, batch sizes for training and evaluation, number of training epochs, weight decay for regularization, and logging settings.

In [ ]:
# Preparing the Data Collator and Training Arguments
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

The compute_metrics function calculates the accuracy of the model's predictions. It takes the output of the model predictions and computes the predicted class labels by taking the argmax of the logits. It then compares these predictions to the true labels (p.label_ids) to calculate the accuracy score using accuracy_score from scikit-learn.

In [ ]:
# Defining the compute_metrics Function
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, predictions)}

The Trainer class is instantiated with the model, training arguments, training dataset, evaluation dataset, data collator, and the metrics computation function. The Trainer class simplifies the training loop by managing the training process, including model updates, evaluation, and logging.

In [ ]:
# Initializing the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Provide the evaluation dataset
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

The training process is initiated using the trainer.train() method. During training, the model learns to minimize the loss by adjusting its weights based on the backpropagation of errors. The model's performance is periodically evaluated on the validation set, and the best model (if applicable) is saved to the specified output directory.

In [ ]:
# Training the Model
trainer.train()

After training, the model's performance is evaluated on the validation set using the trainer.evaluate() method. This provides metrics, including validation accuracy, which is printed to the console.

In [ ]:
# Evaluating the Model
eval_results = trainer.evaluate()
print(f"Validation Accuracy: {eval_results.get('eval_accuracy', 'N/A'):.4f}")

Predictions are made on the evaluation dataset, and the resulting predicted labels (y_pred) are compared with the true labels (y_true). A confusion matrix is computed and visualized using Seaborn to show the number of correct and incorrect predictions for each class. This visualization helps to understand the model's performance in detail. Additionally, a classification report is generated using classification_report from scikit-learn, providing precision, recall, F1-score, and support for each class. This report is printed to give a comprehensive view of the model's classification performance.

In [ ]:
# Predicting from the Evaluation Dataset
predictions = trainer.predict(eval_dataset)

In [ ]:
# Predicted Labels and True Labels
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

In [ ]:
# Computing the Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
# Displaying the Confusion Matrix Using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(4), yticklabels=range(4))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Generating a Classification Report
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3'])
print(report)

In [ ]:
# Classifying the specific sentences and comparing with true labels
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

A text classification pipeline is created using the fine-tuned model and tokenizer. This pipeline allows easy application of the model to new sentences. The provided sentences are classified, and the predicted labels are output. These predicted labels are then compared with the true labels, which are either "Sci-Tech Annotation" or "Business Annotation." The true and predicted labels for each sentence are printed, demonstrating the model's real-world application and effectiveness in categorizing new inputs.

In [ ]:
# List of sentences
sentences = [
    {'text': 'The Politics of Time and Dispossession Make a commitment to subscribe, donate and/or place all of your book and other product orders from Amazon.com and others through MMN Shopping web-site by clicking here.'},
    {'text': 'TDK launches 400GB data cartridge TDK will start selling new 400GB data storage cartridges based on the LTO (Linear Tape-Open) Ultrium 3 format in Japan and elsewhere from the middle of November.'},
    {'text': 'With Charley on Their Minds, Floridians Brace for Round 2 Al Cialella is from Philadelphia, where he always thought of quot;sturdy quot; oak trees as symbols of strength. He never knew that in central Florida, the water table '},
    {'text': 'Wealth in America In what has become an annual ritual, Forbes Magazine has released its "400 Richest in America", a list of the wealthiest in a land which is simultaneously revered and reviled for its wealth. Between the lines are some suprisingly revealing truths about what it takes to be rich in America.'},
    {'text': 'Court Deals Blow to Anti-Piracy Efforts (Reuters) Reuters - A federal appeals court on Thursday\\delivered a stinging blow to the anti-piracy efforts of major\\movie studios and music companies, ruling that several major\\online file-sharing software companies are not liable for\\copyright infringement.'},
    {'text': '2 Vietnamese die of unidentified virus Among three recent human deaths in Vietnam #39;s northern Thai Nguyen province, two were initially reported to have contracted an unidentified dangerous virus, local newspaper Labor reported Monday.'}
]

In [ ]:
# True Labels
true_labels = [
    'Business Annotation',
    'Sci-Tech Annotation',
    'Other',
    'Business Annotation',
    'Sci-Tech Annotation',
    'Sci-Tech Annotation'
]